<a href="https://colab.research.google.com/github/DavoodSZ1993/Dive_into_Deep_Learning/blob/main/4_2the_image_classification_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install d2l==1.0.0-alpha1.post0

In [9]:
%matplotlib inline
import time
import torch
import torchvision
from torchvision import transforms
from matplotlib_inline import backend_inline
import collections
import inspect


In [10]:
def use_svg_display():
  """Use the svg (Scalable Vector Graphics) format to display a plot in Jupyter."""
  backend_inline.set_matplotlib_formats('svg')

use_svg_display()

## 1. Object-oriented Design for Implementation

In [11]:
import numpy as np
from torch import nn
from d2l import torch as d2l

### 1.1 Utilities

In [12]:
def add_to_class(Class):
  def wrapper(obj):
    setattr(Class, obj.__name__, obj)
  return wrapper

In [13]:
class A:
  def __init__(self):
    self.b = 1

a = A()

In [14]:
@add_to_class(A)
def do(self):
  print('Class attribute "b" is', self.b)

a.do()

Class attribute "b" is 1


In [15]:
class Hyperparameters:
  def save_hyperparameters(self, ignore=[]):
    """Save function arguments into class attributes."""
    frame = inspect.currentframe().f_back
    _, _, _, local_vars = inspect.getargvalues(frame)
    self.hparams = {k:v for k, v in local_vars.items() if k not in set(ignore+['self']) and not k.startswith('_')}
    for k, v in self.hparams.items():
      setattr(self, k, v)

In [16]:
# Call the fully implemented GyperParameters class saved in d2l
class B(Hyperparameters):
  def __init__(self, a, b, c):
    self.save_hyperparameters(ignore=['c'])
    print('self.a=', self.a, 'self.b=', self.b)
    print('There is no self.c=', not hasattr(self, 'c'))

In [17]:
b = B(a=1, b=2, c=3)

self.a= 1 self.b= 2
There is no self.c= True
